In [9]:
from yarc import CBA
from yarc.Structure import TransactionDB
from yarc.Mine_Classi_Alg.generating_CARS import ClassAssocationRule, Antecedent, Consequent, top_rules, CARlist
from yarc.Mine_Classi_Alg.m2classi import M2Classi
from yarc.Mine_Classi_Alg.predictor import Predictor
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from yarc.qcba.data_structures import *
from yarc.qcba import *
from yarc.qcba.transformation import *
import Orange
from sklearn.preprocessing import LabelEncoder
from Orange.data.pandas_compat import table_from_frame,table_to_frame


In [17]:
def discretiseRule(X):
    temp = Orange.data.Table(X)
    disc = Orange.preprocess.Discretize()
    disc.method = Orange.preprocess.discretize.EqualFreq(n=3)
    d_temp = disc(temp)
    X= table_to_frame(d_temp)
    return X

def runQCBA(X,y,target):
    X = discretiseRule(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=25, test_size=0.2, stratify = y)#25
    train = pd.concat([X_train, y_train], axis=1)
    test=pd.concat([X_test, y_test], axis=1)
    
    txns = TransactionDB.from_DataFrame(train, target=target)
    txnstest = TransactionDB.from_DataFrame(test, target=target)
    
    cba=CBA()
    cba.fit(txns)
    acc1=cba.rule_model_accuracy(txnstest) #Part 2 model
    # get the best association rules
    rules = top_rules(txns.string_representation)
    # convert them to class association rules
    cars = CARlist(rules)
    predictor = M2Classi(cars, txns).build()
    acc2= predictor.test_transactions(txnstest)
    ds = test.reset_index() #test set of undis
    quant_dataset = QuantitativeDataFrame(ds)
    
    Y = ds[target]
    rules=cba.pre.rules
    quant_rules = [ QuantitativeCAR(r) for r in rules ] #rules of undis
    
    rules2 = predictor.rules
    quant_rules2 = [ QuantitativeCAR(r) for r in rules2 ] #rules of undis
    
    qcba_transformation = QCBATransformation(quant_dataset)
    refitted_rules = qcba_transformation.refitter.transform(quant_rules)
    literal_pruned_rules = qcba_transformation.literal_pruner.transform(refitted_rules)
    trimmed_rules = qcba_transformation.trimmer.transform(literal_pruned_rules)
    pruned_rules, default_class = qcba_transformation.post_pruner.transform(trimmed_rules)
    refitted_rules2 = qcba_transformation.refitter.transform(quant_rules2)
    literal_pruned_rules2 = qcba_transformation.literal_pruner.transform(refitted_rules2)
    trimmed_rules2 = qcba_transformation.trimmer.transform(literal_pruned_rules2)
    pruned_rules2, default_class2 = qcba_transformation.post_pruner.transform(trimmed_rules2)
    q_clf2 = QuantitativeClassifier(pruned_rules, default_class)
    acc3=q_clf2.rule_model_accuracy(quant_dataset, Y) #Part 5 model
    q_clf1 = QuantitativeClassifier(pruned_rules2, default_class2)
    acc4=q_clf1.rule_model_accuracy(quant_dataset, Y) #Part 5 model(top rules)
    print("Accuracy of CBA:",acc1)
    print("Accuracy of CBA (Top k):",acc2)
    print("Accuracy of QCBA:",acc3)
    print("Accuracy of QCBA (Top k):",acc4)

# Iris

In [18]:
iris = pd.read_csv ('iris.csv')
X=iris[["sepallength","petalwidth","sepalwidth","petallength"]]
y=iris[["class"]]
runQCBA(X,y,"class")

Accuracy of CBA: 0.9
Accuracy of CBA (Top k): 0.9
Accuracy of QCBA: 0.9333333333333333
Accuracy of QCBA (Top k): 0.9333333333333333


# Wave

In [19]:
wave = pd.read_csv ('wave.csv')
wave = wave.dropna()
X=wave[['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20']]
y=wave[["class"]]
runQCBA(X,y,"class")

Accuracy of CBA: 0.764
Accuracy of CBA (Top k): 0.797
Accuracy of QCBA: 0.768
Accuracy of QCBA (Top k): 0.81


# Heart

In [20]:
heart = pd.read_csv ('heart.csv')
heart = heart.dropna()
X=heart[["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]]
y=heart[["class"]]
runQCBA(X,y,"class")

Accuracy of CBA: 0.8148148148148148
Accuracy of CBA (Top k): 0.7777777777777778
Accuracy of QCBA: 0.9259259259259259
Accuracy of QCBA (Top k): 0.8703703703703703


# Breast Cancer

In [21]:
from sklearn.preprocessing import OrdinalEncoder
breastCancer = pd.read_csv ('breastCancer.csv')
breastCancer = breastCancer.dropna()
X=breastCancer[["menopause", "age", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]]
#encoding the normal data
for col in ["menopause","node-caps","breast","breast-quad","irradiat"]:
   X[col] = LabelEncoder().fit_transform(X[col])
#encoding the ordinal data #Ordinal Encoder not working so i just hardcoded first
ordinalData = ["age","tumor-size","inv-nodes"]
for i in ordinalData:
   columns = X[i].unique()
   columns.sort() #sorting the labels
   for j in range(len(columns)):
      X.loc[X[i]==columns[j],i] = j
y=breastCancer[["class"]]
runQCBA(X,y,"class")

C:\Users\theco\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\theco\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Accuracy of CBA: 0.6491228070175439
Accuracy of CBA (Top k): 0.6666666666666666
Accuracy of QCBA: 0.8070175438596491
Accuracy of QCBA (Top k): 0.7894736842105263


# German

In [22]:
german = pd.read_csv ('german.csv')
german = german.dropna()
X=german[["checkAccStatus", "durationMth", "credHist", "purpose", "credAmt","savAccBond","emplySince","instRate","personalStatSex","otherDebtGuar","presResSince","prpty","age(years)","otherInstallPlans","housing","numExistCreds","job","numPplMaintain","telephone","frgnWorker"]]
for col in ['checkAccStatus','credHist','purpose','savAccBond','emplySince','personalStatSex','otherDebtGuar','prpty','otherInstallPlans','housing','job','telephone','frgnWorker']:
    X[col] = LabelEncoder().fit_transform(X[col])
y=german[["goodBad"]]
runQCBA(X,y,"goodBad")

C:\Users\theco\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Accuracy of CBA: 0.7
Accuracy of CBA (Top k): 0.73
Accuracy of QCBA: 0.745
Accuracy of QCBA (Top k): 0.76


# University Student Placement

In [24]:
campusPlacement = pd.read_csv("Placement_Data_Full_Class.csv")
campusPlacement.head()
campusPlacement = campusPlacement.drop(['salary'],1)
campusPlacement = campusPlacement.dropna()
X = campusPlacement[['sl_no', 'gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s', 'degree_p', 'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p']]
#creating a function to encode categorical features into numerical
for col in ['gender','ssc_b','hsc_b','hsc_s','degree_t','workex','specialisation']:
    X[col] = LabelEncoder().fit_transform(X[col])
y=campusPlacement[['status']]
runQCBA(X,y,"status")

C:\Users\theco\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\theco\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Accuracy of CBA: 0.7674418604651163
Accuracy of CBA (Top k): 0.7674418604651163
Accuracy of QCBA: 0.9069767441860465
Accuracy of QCBA (Top k): 0.8604651162790697


# Stroke

In [25]:
stroke = pd.read_csv('healthcare-dataset-stroke-data.csv')
stroke = stroke.drop(['id'],1)
stroke = stroke.dropna()
X=stroke[["gender", "age", "heart_disease", "ever_married", "work_type","Residence_type","avg_glucose_level","bmi","smoking_status"]]
#encoding the normal data
for col in ["gender","ever_married","work_type","Residence_type", "smoking_status"]:
   X[col] = LabelEncoder().fit_transform(X[col])
y=stroke[["stroke"]]
runQCBA(X,y,"stroke")

C:\Users\theco\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
C:\Users\theco\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Accuracy of CBA: 0.534698521046644
Accuracy of CBA (Top k): 0.534698521046644
Accuracy of QCBA: 0.534698521046644
Accuracy of QCBA (Top k): 0.8623435722411832


In [ ]:
txnstest = TransactionDB.from_DataFrame(test, target="class") 

df2=pd.read_csv("german.csv")
df3=pd.read_csv("heart.csv")
df4=pd.read_csv("breastCancer.csv")
df5=pd.read_csv("wave.csv")
df6=pd.read_csv("healthcare-dataset-stroke-data.csv")
df7=pd.read_csv("Placement_Data_Full_Class.csv")

labelencoder = LabelEncoder()
df2['goodBad'] = labelencoder.fit_transform(df2['goodBad'])
df2['checkAccStatus'] = labelencoder.fit_transform(df2['checkAccStatus'])
df2['credHist'] = labelencoder.fit_transform(df2['credHist'])
df2['purpose'] = labelencoder.fit_transform(df2['purpose'])
df2['savAccBond'] = labelencoder.fit_transform(df2['savAccBond'])
df2['emplySince'] = labelencoder.fit_transform(df2['emplySince'])
df2['personalStatSex'] = labelencoder.fit_transform(df2['personalStatSex'])
df2['otherDebtGuar'] = labelencoder.fit_transform(df2['otherDebtGuar'])
df2['prpty'] = labelencoder.fit_transform(df2['prpty'])
df2['otherInstallPlans'] = labelencoder.fit_transform(df2['otherInstallPlans'])
df2['housing'] = labelencoder.fit_transform(df2['housing'])
df2['job'] = labelencoder.fit_transform(df2['job'])
df2['telephone'] = labelencoder.fit_transform(df2['telephone'])
df2['frgnWorker'] = labelencoder.fit_transform(df2['frgnWorker'])
df4['age'] = labelencoder.fit_transform(df4['age'])
df4['menopause'] = labelencoder.fit_transform(df4['menopause'])
df4['tumor-size'] = labelencoder.fit_transform(df4['tumor-size'])
df4['inv-nodes'] = labelencoder.fit_transform(df4['inv-nodes'])
df4['node-caps'] = labelencoder.fit_transform(df4['node-caps'])
df4['breast'] = labelencoder.fit_transform(df4['breast'])
df4['breast'] = labelencoder.fit_transform(df4['breast'])
df4['breast-quad'] = labelencoder.fit_transform(df4['breast-quad'])
df4['irradiat'] = labelencoder.fit_transform(df4['irradiat'])
df6['gender'] = labelencoder.fit_transform(df6['gender'])
df6['ever_married'] = labelencoder.fit_transform(df6['ever_married'])
df6['work_type'] = labelencoder.fit_transform(df6['work_type'])
df6['Residence_type'] = labelencoder.fit_transform(df6['Residence_type'])
df6['smoking_status'] = labelencoder.fit_transform(df6['smoking_status'])
df7['specialisation'] = labelencoder.fit_transform(df7['specialisation'])
df7['workex'] = labelencoder.fit_transform(df7['workex'])
df7['degree_t'] = labelencoder.fit_transform(df7['degree_t'])
df7['hsc_s'] = labelencoder.fit_transform(df7['hsc_s'])
df7['hsc_b'] = labelencoder.fit_transform(df7['hsc_b'])
df7['ssc_b'] = labelencoder.fit_transform(df7['ssc_b'])
df7['gender'] = labelencoder.fit_transform(df7['gender'])
df7=df7.drop(["salary","sl_no"], axis=1)


X2=df2[['checkAccStatus', 'durationMth', 'credHist', 'purpose', 'credAmt',
       'savAccBond', 'emplySince', 'instRate', 'personalStatSex',
       'otherDebtGuar', 'presResSince', 'prpty', 'age(years)',
       'otherInstallPlans', 'housing', 'numExistCreds', 'job',
       'numPplMaintain', 'telephone', 'frgnWorker']]
X2 = discretiseRule(X2)
y2=df2[["goodBad"]]
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=25, test_size=0.2, stratify = y2)#25
train2 = pd.concat([X2_train, y2_train], axis=1)
test2=pd.concat([X2_test, y2_test], axis=1)
txns2 = TransactionDB.from_DataFrame(train2, target="goodBad")
txnstest2 = TransactionDB.from_DataFrame(test2, target="goodBad") 

X3=df3[['age', 'sex', 'chest pain type', 'resting blood pressure',
       'serum cholesterol (mg/dl)', 'resting blood sugar >120mg/dl',
       'resting electrocariographic results', 'maximum heart rate received',
       'exercise induced angina', 'oldpeak', 'slopePeak', 'numMajorVessels',
       'thal']]
X3 = discretiseRule(X3)
y3=df3[["class"]]
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, random_state=25, test_size=0.2, stratify = y3)#25
train3 = pd.concat([X3_train, y3_train], axis=1)
test3=pd.concat([X3_test, y3_test], axis=1)
txns3 = TransactionDB.from_DataFrame(train3, target="class")
txnstest3 = TransactionDB.from_DataFrame(test3, target="class") 

X4=df4[[ 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps',
       'deg-malig', 'breast', 'breast-quad', 'irradiat']]
X4 = discretiseRule(X4)
y4=df4[['class']]
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, random_state=25, test_size=0.2, stratify = y4)#25
train4 = pd.concat([X4_train, y4_train], axis=1)
test4=pd.concat([X4_test, y4_test], axis=1)
txns4 = TransactionDB.from_DataFrame(train4, target="class")
txnstest4 = TransactionDB.from_DataFrame(test4, target="class") 

X5=df5[['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10',
       'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20']]
X5 = discretiseRule(X5)
y5=df5[["class"]]
X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, random_state=25, test_size=0.2, stratify = y5)#25
train5 = pd.concat([X5_train, y5_train], axis=1)
test5=pd.concat([X5_test, y5_test], axis=1)
txns5 = TransactionDB.from_DataFrame(train5, target="class")
txnstest5 = TransactionDB.from_DataFrame(test5, target="class") 


X6=df6[['id',
 'gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status']]
X6 = discretiseRule(X6)
y6=df6[['stroke']]
X6_train, X6_test, y6_train, y6_test = train_test_split(X6, y6, random_state=25, test_size=0.2, stratify = y6)#25
train6 = pd.concat([X6_train, y6_train], axis=1)
test6=pd.concat([X6_test, y6_test], axis=1)
txns6 = TransactionDB.from_DataFrame(train6, target="stroke")
txnstest6 = TransactionDB.from_DataFrame(test6, target="stroke") 

X7=df7[['gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s',
       'degree_p', 'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p']]
X7= discretiseRule(X7)
y7=df7[['status']]
X7_train, X7_test, y7_train, y7_test = train_test_split(X7, y7, random_state=25, test_size=0.2, stratify = y7)#25
train7 = pd.concat([X7_train, y7_train], axis=1)
test7=pd.concat([X7_test, y7_test], axis=1)
txns7 = TransactionDB.from_DataFrame(train7, target="status")
txnstest7 = TransactionDB.from_DataFrame(test7, target="status") 

In [ ]:
cba = CBA()
train

In [ ]:
cba.fit(txns)
cba.rule_model_accuracy(txnstest) #Part 2 model


In [ ]:
# get the best association rules
rules = top_rules(txns.string_representation)

# convert them to class association rules
cars = CARlist(rules)

predictor = M2Classi(cars, txns).build()

accuracy = predictor.test_transactions(txnstest)
accuracy

In [ ]:
ds = test.reset_index() #test set of undis
quant_dataset = QuantitativeDataFrame(ds)
Y = ds["class"]
quant_dataset.dataframe.head()

In [ ]:
rules=cba.pre.rules
quant_rules = [ QuantitativeCAR(r) for r in rules ] #rules of undis
rules

In [ ]:
rules2 = predictor.rules
quant_rules2 = [ QuantitativeCAR(r) for r in rules2 ] #rules of undis
rules2

In [ ]:
qcba_transformation = QCBATransformation(quant_dataset)
quant_dataset.dataframe["class"]
quant_dataset._QuantitativeDataFrame__preprocessed_columns

In [ ]:
refitted_rules = qcba_transformation.refitter.transform(quant_rules)
literal_pruned_rules = qcba_transformation.literal_pruner.transform(refitted_rules)
trimmed_rules = qcba_transformation.trimmer.transform(literal_pruned_rules)
pruned_rules, default_class = qcba_transformation.post_pruner.transform(trimmed_rules)

In [ ]:
refitted_rules2 = qcba_transformation.refitter.transform(quant_rules2)
literal_pruned_rules2 = qcba_transformation.literal_pruner.transform(refitted_rules2)
trimmed_rules2 = qcba_transformation.trimmer.transform(literal_pruned_rules2)
pruned_rules2, default_class2 = qcba_transformation.post_pruner.transform(trimmed_rules2)

In [ ]:
q_clf2 = QuantitativeClassifier(pruned_rules, default_class)
q_clf2.rule_model_accuracy(quant_dataset, Y) #Part 5 model

In [ ]:
q_clf1 = QuantitativeClassifier(pruned_rules2, default_class2)
q_clf1.rule_model_accuracy(quant_dataset, Y) #Part 5 model

In [ ]:
cba=CBA()
cba.fit(txns2)
acc1=cba.rule_model_accuracy(txnstest2) #Part 2 model
# get the best association rules
rules = top_rules(txns2.string_representation)
# convert them to class association rules
cars = CARlist(rules)
predictor = M2Classi(cars, txns2).build()
acc2= predictor.test_transactions(txnstest2)
ds = test2.reset_index() #test set of undis
quant_dataset = QuantitativeDataFrame(ds)
Y = ds["goodBad"]
rules=cba.pre.rules
quant_rules = [ QuantitativeCAR(r) for r in rules ] #rules of undis
rules2 = predictor.rules
quant_rules2 = [ QuantitativeCAR(r) for r in rules2 ] #rules of undis
qcba_transformation = QCBATransformation(quant_dataset)
refitted_rules = qcba_transformation.refitter.transform(quant_rules)
literal_pruned_rules = qcba_transformation.literal_pruner.transform(refitted_rules)
trimmed_rules = qcba_transformation.trimmer.transform(literal_pruned_rules)
pruned_rules, default_class = qcba_transformation.post_pruner.transform(trimmed_rules)
refitted_rules2 = qcba_transformation.refitter.transform(quant_rules2)
literal_pruned_rules2 = qcba_transformation.literal_pruner.transform(refitted_rules2)
trimmed_rules2 = qcba_transformation.trimmer.transform(literal_pruned_rules2)
pruned_rules2, default_class2 = qcba_transformation.post_pruner.transform(trimmed_rules2)
q_clf2 = QuantitativeClassifier(pruned_rules, default_class)
acc3=q_clf2.rule_model_accuracy(quant_dataset, Y) #Part 5 model
q_clf1 = QuantitativeClassifier(pruned_rules2, default_class2)
acc4=q_clf1.rule_model_accuracy(quant_dataset, Y) #Part 5 model(top rules)
print(acc1,acc2,acc3,acc4)

In [ ]:
cba=CBA()
cba.fit(txns3)
acc1=cba.rule_model_accuracy(txnstest3) #Part 2 model
# get the best association rules
rules = top_rules(txns3.string_representation)
# convert them to class association rules
cars = CARlist(rules)
predictor = M2Classi(cars, txns3).build()
acc2= predictor.test_transactions(txnstest3)
ds = test3.reset_index() #test set of undis
quant_dataset = QuantitativeDataFrame(ds)
Y = ds["class"]
rules=cba.pre.rules
quant_rules = [ QuantitativeCAR(r) for r in rules ] #rules of undis
rules2 = predictor.rules
quant_rules2 = [ QuantitativeCAR(r) for r in rules2 ] #rules of undis
qcba_transformation = QCBATransformation(quant_dataset)
refitted_rules = qcba_transformation.refitter.transform(quant_rules)
literal_pruned_rules = qcba_transformation.literal_pruner.transform(refitted_rules)
trimmed_rules = qcba_transformation.trimmer.transform(literal_pruned_rules)
pruned_rules, default_class = qcba_transformation.post_pruner.transform(trimmed_rules)
refitted_rules2 = qcba_transformation.refitter.transform(quant_rules2)
literal_pruned_rules2 = qcba_transformation.literal_pruner.transform(refitted_rules2)
trimmed_rules2 = qcba_transformation.trimmer.transform(literal_pruned_rules2)
pruned_rules2, default_class2 = qcba_transformation.post_pruner.transform(trimmed_rules2)
q_clf2 = QuantitativeClassifier(pruned_rules, default_class)
acc3=q_clf2.rule_model_accuracy(quant_dataset, Y) #Part 5 model
q_clf1 = QuantitativeClassifier(pruned_rules2, default_class2)
acc4=q_clf1.rule_model_accuracy(quant_dataset, Y) #Part 5 model(top rules)
print(acc1,acc2,acc3,acc4)

In [ ]:
cba=CBA()
cba.fit(txns4)
acc1=cba.rule_model_accuracy(txnstest4) #Part 2 model
# get the best association rules
rules = top_rules(txns4.string_representation)
# convert them to class association rules
cars = CARlist(rules)
predictor = M2Classi(cars, txns4).build()
acc2= predictor.test_transactions(txnstest4)
ds = test4.reset_index() #test set of undis
quant_dataset = QuantitativeDataFrame(ds)
Y = ds["class"]
rules=cba.pre.rules
quant_rules = [ QuantitativeCAR(r) for r in rules ] #rules of undis
rules2 = predictor.rules
quant_rules2 = [ QuantitativeCAR(r) for r in rules2 ] #rules of undis
qcba_transformation = QCBATransformation(quant_dataset)
refitted_rules = qcba_transformation.refitter.transform(quant_rules)
literal_pruned_rules = qcba_transformation.literal_pruner.transform(refitted_rules)
trimmed_rules = qcba_transformation.trimmer.transform(literal_pruned_rules)
pruned_rules, default_class = qcba_transformation.post_pruner.transform(trimmed_rules)
refitted_rules2 = qcba_transformation.refitter.transform(quant_rules2)
literal_pruned_rules2 = qcba_transformation.literal_pruner.transform(refitted_rules2)
trimmed_rules2 = qcba_transformation.trimmer.transform(literal_pruned_rules2)
pruned_rules2, default_class2 = qcba_transformation.post_pruner.transform(trimmed_rules2)
q_clf2 = QuantitativeClassifier(pruned_rules, default_class)
acc3=q_clf2.rule_model_accuracy(quant_dataset, Y) #Part 5 model
q_clf1 = QuantitativeClassifier(pruned_rules2, default_class2)
acc4=q_clf1.rule_model_accuracy(quant_dataset, Y) #Part 5 model(top rules)
print(acc1,acc2,acc3,acc4)

In [ ]:
cba=CBA()
cba.fit(txns5)
acc1=cba.rule_model_accuracy(txnstest5) #Part 2 model
# get the best association rules
rules = top_rules(txns5.string_representation)
# convert them to class association rules
cars = CARlist(rules)
predictor = M2Classi(cars, txns5).build()
acc2= predictor.test_transactions(txnstest5)
ds = test5.reset_index() #test set of undis
quant_dataset = QuantitativeDataFrame(ds)
Y = ds["class"]
rules=cba.pre.rules
quant_rules = [ QuantitativeCAR(r) for r in rules ] #rules of undis
rules2 = predictor.rules
quant_rules2 = [ QuantitativeCAR(r) for r in rules2 ] #rules of undis
qcba_transformation = QCBATransformation(quant_dataset)
refitted_rules = qcba_transformation.refitter.transform(quant_rules)
literal_pruned_rules = qcba_transformation.literal_pruner.transform(refitted_rules)
trimmed_rules = qcba_transformation.trimmer.transform(literal_pruned_rules)
pruned_rules, default_class = qcba_transformation.post_pruner.transform(trimmed_rules)
refitted_rules2 = qcba_transformation.refitter.transform(quant_rules2)
literal_pruned_rules2 = qcba_transformation.literal_pruner.transform(refitted_rules2)
trimmed_rules2 = qcba_transformation.trimmer.transform(literal_pruned_rules2)
pruned_rules2, default_class2 = qcba_transformation.post_pruner.transform(trimmed_rules2)
q_clf2 = QuantitativeClassifier(pruned_rules, default_class)
acc3=q_clf2.rule_model_accuracy(quant_dataset, Y) #Part 5 model
q_clf1 = QuantitativeClassifier(pruned_rules2, default_class2)
acc4=q_clf1.rule_model_accuracy(quant_dataset, Y) #Part 5 model(top rules)
print(acc1,acc2,acc3,acc4)

In [ ]:
cba=CBA()
cba.fit(txns6)
acc1=cba.rule_model_accuracy(txnstest6) #Part 2 model
# get the best association rules
rules = top_rules(txns6.string_representation)
# convert them to class association rules
cars = CARlist(rules)
predictor = M2Classi(cars, txns6).build()
acc2= predictor.test_transactions(txnstest6)
ds = test6.reset_index() #test set of undis
quant_dataset = QuantitativeDataFrame(ds)
Y = ds["stroke"]
rules=cba.pre.rules
quant_rules = [ QuantitativeCAR(r) for r in rules ] #rules of undis
rules2 = predictor.rules
quant_rules2 = [ QuantitativeCAR(r) for r in rules2 ] #rules of undis
qcba_transformation = QCBATransformation(quant_dataset)
refitted_rules = qcba_transformation.refitter.transform(quant_rules)
literal_pruned_rules = qcba_transformation.literal_pruner.transform(refitted_rules)
trimmed_rules = qcba_transformation.trimmer.transform(literal_pruned_rules)
pruned_rules, default_class = qcba_transformation.post_pruner.transform(trimmed_rules)
refitted_rules2 = qcba_transformation.refitter.transform(quant_rules2)
literal_pruned_rules2 = qcba_transformation.literal_pruner.transform(refitted_rules2)
trimmed_rules2 = qcba_transformation.trimmer.transform(literal_pruned_rules2)
pruned_rules2, default_class2 = qcba_transformation.post_pruner.transform(trimmed_rules2)
q_clf2 = QuantitativeClassifier(pruned_rules, default_class)
acc3=q_clf2.rule_model_accuracy(quant_dataset, Y) #Part 5 model
q_clf1 = QuantitativeClassifier(pruned_rules2, default_class2)
acc4=q_clf1.rule_model_accuracy(quant_dataset, Y) #Part 5 model(top rules)
print(acc1,acc2,acc3,acc4)

In [ ]:
cba=CBA()
cba.fit(txns7)
acc1=cba.rule_model_accuracy(txnstest7) #Part 2 model
# get the best association rules
rules = top_rules(txns7.string_representation)
# convert them to class association rules
cars = CARlist(rules)
predictor = M2Classi(cars, txns7).build()
acc2= predictor.test_transactions(txnstest7)
ds = test7.reset_index() #test set of undis
quant_dataset = QuantitativeDataFrame(ds)
Y = ds["status"]
rules=cba.pre.rules
quant_rules = [ QuantitativeCAR(r) for r in rules ] #rules of undis
rules2 = predictor.rules
quant_rules2 = [ QuantitativeCAR(r) for r in rules2 ] #rules of undis
qcba_transformation = QCBATransformation(quant_dataset)
refitted_rules = qcba_transformation.refitter.transform(quant_rules)
literal_pruned_rules = qcba_transformation.literal_pruner.transform(refitted_rules)
trimmed_rules = qcba_transformation.trimmer.transform(literal_pruned_rules)
pruned_rules, default_class = qcba_transformation.post_pruner.transform(trimmed_rules)
refitted_rules2 = qcba_transformation.refitter.transform(quant_rules2)
literal_pruned_rules2 = qcba_transformation.literal_pruner.transform(refitted_rules2)
trimmed_rules2 = qcba_transformation.trimmer.transform(literal_pruned_rules2)
pruned_rules2, default_class2 = qcba_transformation.post_pruner.transform(trimmed_rules2)
q_clf2 = QuantitativeClassifier(pruned_rules, default_class)
acc3=q_clf2.rule_model_accuracy(quant_dataset, Y) #Part 5 model
q_clf1 = QuantitativeClassifier(pruned_rules2, default_class2)
acc4=q_clf1.rule_model_accuracy(quant_dataset, Y) #Part 5 model(top rules)
print(acc1,acc2,acc3,acc4)